# Recommending Locations for an Italian Restaurant in Fort Worth

In [20]:
# package imports
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Data Collection

### Demographics

In [21]:
# API Endpoints for Fort Worth and National Data
fort_worth_url = "https://api.censusreporter.org/1.0/data/show/latest?geo_ids=16000US4827000&table_ids=B01003,B19013,B01002"
us_url = "https://api.censusreporter.org/1.0/data/show/latest?geo_ids=01000US&table_ids=B01003,B19013,B01002"

# Fetch data
fort_worth_data = requests.get(fort_worth_url).json()
us_data = requests.get(us_url).json()

# Extract key statistics
fort_worth_geo = fort_worth_data["data"]["16000US4827000"]
us_geo = us_data["data"]["01000US"]

# Land area values (Census QuickFacts)
fort_worth_land_area = 342.2  # Fort Worth land area (sq. miles)
us_land_area = 3531905  # US total land area (sq. miles)

# Calculate population density
fort_worth_population = fort_worth_geo["B01003"]["estimate"]["B01003001"]
us_population = us_geo["B01003"]["estimate"]["B01003001"]

fort_worth_density = fort_worth_population / fort_worth_land_area
us_density = us_population / us_land_area

# Convert to DataFrame
df = pd.DataFrame({
    "Metric": ["Total Population", "Median Household Income", "Median Age", "Population Density (per sq. mile)"],
    "Fort Worth": [
        fort_worth_population,  # Fort Worth Population
        fort_worth_geo["B19013"]["estimate"]["B19013001"],  # Fort Worth Median Income
        fort_worth_geo["B01002"]["estimate"]["B01002001"],  # Fort Worth Median Age
        round(fort_worth_density, 2)  # Fort Worth Population Density
    ],
    "National Average": [
        us_population,  # US Population
        us_geo["B19013"]["estimate"]["B19013001"],  # US Median Income
        us_geo["B01002"]["estimate"]["B01002001"],  # US Median Age
        round(us_density, 2)  # US Population Density
    ]
})

# Display comparison table
print(df)


                              Metric  Fort Worth  National Average
0                   Total Population   976932.00      3.349149e+08
1            Median Household Income    77082.00      7.771900e+04
2                         Median Age       33.70      3.920000e+01
3  Population Density (per sq. mile)     2854.86      9.483000e+01
